In [1]:
import os
import altair as alt


#path = os.path.join(local_data, 'VerenigingHogescholen/2018/gegevensset 2018/flatfiles', 'voortgang_2018.csv')
path = os.path.join(local_data, 'VerenigingHogescholen/2018/gegevensset 2018/flatfiles', 'instroom_detail_2018.csv')

# inlezen data:
df = pd.read_csv(path
                , encoding='latin1'
                , sep='|')



In [22]:
instroom_5jaar = df[df.inschr_jaar > 2013].copy().dropna()

In [23]:
scholen = instroom_5jaar[['vooropleiding_voorho_inst_naam','vooropleiding_voorho_inst_gemeente']].drop_duplicates()
len(scholen)

761

In [24]:
scholen

,vooropleiding_voorho_inst_naam,vooropleiding_voorho_inst_gemeente
1324428,deltion college,zwolle
1324429,wellantcollege,houten
1324430,aeres (v) mbo,ede
1324433,almere college,kampen
1324437,landstede,zwolle
...,...,...
1868137,"loc, morgen college",harderwijk
1868291,chr sgm veenendaal,veenendaal
1869114,stg cheider vo,amsterdam
1873057,inspecteur boelensschool,schiermonnikoog


In [25]:
import googlemaps
from datetime import datetime

def _get_distance_between_locations(startpunt, eindpunt, google_api_key=google_api_key):
    """Haalt reisafstand en reistijd op bij Google."""
    gmaps = googlemaps.Client(key=google_api_key)
    result = pd.Series()
    try:
        googleresult = gmaps.distance_matrix(origins=startpunt
                     , destinations=eindpunt
                     , mode='transit'
                     , language='nl'
                     , units='metrics'
                     , arrival_time=datetime(2019, 11, 19, 8, 30, 0)
                              )
        
        result['OvAfstandMeter'] = googleresult['rows'][0]['elements'][0]['distance']['value']
        result['OvAfstand'] = googleresult['rows'][0]['elements'][0]['distance']['text']
        result['OvReistijdSeconden'] = googleresult['rows'][0]['elements'][0]['duration']['value']
        result['OvReistijd'] = googleresult['rows'][0]['elements'][0]['duration']['text']

        places = gmaps.places(startpunt, language='nl')
        
        result['Latitude']=places['results'][0]['geometry']['location']['lat']
        result['Longitude']=places['results'][0]['geometry']['location']['lng']
        result['AdresInstelling']=places['results'][0]['formatted_address']
        result['NaamInstellingVolgensGoogle'] = places['results'][0]['name']
    
    except:
        result['StatusGoogleAfstand'] = 'Er ging iets mis'
    finally:
        return result

    
def afstand_en_reistijd(row, kolommen=['vooropleiding_voorho_inst_naam','vooropleiding_voorho_inst_gemeente'], thuisinstelling='Hogeschool Leiden, Leiden'):
    startpunt_list = row[kolommen].values.tolist()
    startpunt = ', '.join(startpunt_list)
    eindpunt = thuisinstelling
    google_data = _get_distance_between_locations(startpunt, eindpunt)
    result = pd.concat([row, google_data])
    return result

In [26]:
scholen.head().apply(afstand_en_reistijd, thuisinstelling='Hogeschool Leiden, Leiden', axis=1)

,vooropleiding_voorho_inst_naam,vooropleiding_voorho_inst_gemeente,OvAfstandMeter,OvAfstand,OvReistijdSeconden,OvReistijd,Latitude,Longitude,AdresInstelling,NaamInstellingVolgensGoogle
1324428,deltion college,zwolle,143160,143 km,7325,2 uur 2 min.,52.520749,6.084291,"Mozartlaan 15, 8031 AA Zwolle, Nederland",Deltion College
1324429,wellantcollege,houten,59753,"59,8 km",6036,1 uur 41 min.,52.038823,5.162414,"Randhoeve 2, 3992 XH Houten, Nederland",Wellantcollege Houten
1324430,aeres (v) mbo,ede,93215,"93,2 km",7296,2 uur 2 min.,52.018436,5.663627,"Zandlaan 31, 6717 LN Ede, Nederland",Aeres VMBO Ede
1324433,almere college,kampen,125371,125 km,8336,2 uur 19 min.,52.555062,5.887738,"Flevoweg 68, 8265 PL Kampen, Nederland",Almere College VIA
1324437,landstede,zwolle,142022,142 km,7194,2 uur 0 min.,52.503922,6.113766,"Fuchsiastraat 1, 8013 ZC Zwolle, Nederland",mbo Menso Alting Zwolle


In [27]:
# hieronder duurt 10 minuten:
reistijd_en_afstand_tot_hl = scholen.apply(afstand_en_reistijd, thuisinstelling='Hogeschool Leiden, Leiden', axis=1)
reistijd_en_afstand_tot_hl['Hogeschool'] = 'HL'
reistijd_en_afstand_tot_hl.to_csv('~/Desktop/hl_reis_afstand_vh.csv', index=False)

In [28]:
reistijd_en_afstand_tot_hva = scholen.apply(afstand_en_reistijd, thuisinstelling='Hogeschool van Amsterdam, Amsterdam', axis=1)
reistijd_en_afstand_tot_hva['Hogeschool'] = 'HvA'
reistijd_en_afstand_tot_hva.to_csv('~/Desktop/hva_reis_afstand_vh.csv', index=False)

In [29]:
reistijd_en_afstand_tot_hu = scholen.apply(afstand_en_reistijd, thuisinstelling='Hogeschool Utrecht, Utrecht', axis=1)
reistijd_en_afstand_tot_hu['Hogeschool'] = 'HU'
reistijd_en_afstand_tot_hu.to_csv('~/Desktop/hu_reis_afstand_vh.csv', index=False)

In [30]:
reistijd_en_afstand_tot_hr = scholen.apply(afstand_en_reistijd, thuisinstelling='Hogeschool Rotterdam, Rotterdam', axis=1)
reistijd_en_afstand_tot_hr['Hogeschool'] = 'HR'
reistijd_en_afstand_tot_hr.to_csv('~/Desktop/hr_reis_afstand_vh.csv', index=False)

In [31]:
reistijd_en_afstand_tot_hhs = scholen.apply(afstand_en_reistijd, thuisinstelling='De Haagse Hogeschool, Den Haag', axis=1)
reistijd_en_afstand_tot_hhs['Hogeschool'] = 'HHS'
reistijd_en_afstand_tot_hhs.to_csv('~/Desktop/hhs_reis_afstand_vh.csv', index=False)

In [32]:
reistijd_en_afstand_tot_inh = scholen.apply(afstand_en_reistijd, thuisinstelling='Hogeschool Inholland, Haarlem', axis=1)
reistijd_en_afstand_tot_inh['Hogeschool'] = 'INH'
reistijd_en_afstand_tot_inh.to_csv('~/Desktop/inh_ha_reis_afstand_vh.csv', index=False)